In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import gc
import time
import pytorch_lightning as pl
from sklearn.model_selection import KFold


In [3]:
df_train=pd.read_csv("./Cdataset_admat_dgc.txt", sep="\t", index_col=0).T
df_train

,D102100,D102300,D102400,D102500,D103100,D103230,D103285,D103780,D104130,D104300,...,D608232,D608266,D608320,D608437,D608456,D608583,D608622,D608636,D608895,D608907
DB00014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB00035,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB00091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB00104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB00115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB08835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB08896,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DB08901,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
DB08906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_train=pd.read_csv("./Cdataset_admat_dgc.txt", sep="\t", index_col=0).T
# 设置测试集比例
test_ratio = 0.6
interaction=np.array(df_train)

# 获取非零元素的坐标
nonzero_coords = np.nonzero(interaction)
nonzero_count = len(nonzero_coords[0])
test_size = int(np.ceil(test_ratio * nonzero_count))
np.random.seed(666)
test_indices = np.random.choice(nonzero_count, size=test_size, replace=False)
test_coords = (nonzero_coords[0][test_indices], nonzero_coords[1][test_indices])
train_matrix = np.copy(interaction)
train_matrix[test_coords] = 0
df_train=train_matrix
rate_matrix=torch.from_numpy(df_train)
rate_matrix=torch.tensor(df_train).clone()#训练集


In [18]:
df_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
# 测试集
test=np.array(list(zip(test_coords[0],test_coords[1])))
test_pd=pd.DataFrame(test)
test_pd.to_csv('Ctest.csv',index=False)
test_pd


,0,1
0,251,390
1,64,320
2,118,174
3,607,392
4,371,6
...,...,...
1507,608,228
1508,221,200
1509,595,208
1510,272,316


In [20]:
np.save(r'./'+'Crate_sparse.npy',rate_matrix.cpu().numpy())